In [3]:
#ОБРУЧАЛКА
import pandas as pd
#загужаем файл остатков и отметает лишнее
ost_obr=pd.read_csv(r'C:\анаконда\tmp.csv', delimiter=';', encoding='utf-8',low_memory=False,
               usecols=["КодСклада", "НазваниеСклада", "ТоварНо","СерийныйНомер","Проект"])
ost_obr=ost_obr[ost_obr.Проект.isin(['ЗОЛОТОЙ','ОРИОН','ЗОЛОТО','ТАЛАНТ'])].sort_values('КодСклада')
del ost_obr['Проект']

#делим файл на две части:магазы и центральные склады
ost_obr1=ost_obr[ost_obr.НазваниеСклада.str.contains('центральный склад|опт',case = False)]
ost_obr=ost_obr[~ost_obr.НазваниеСклада.str.contains('центральный склад|опт',case = False)]
ost_obr=ost_obr[['КодСклада','ТоварНо']]
ost_obr1=ost_obr1[['СерийныйНомер','ТоварНо']]

#обработка заборников верхние пустые строки в загружаемом файле должны быть удалены
zab=pd.read_excel(r'C:\анаконда\zab.xlsx')
zab=zab[["Серийный Но.","Товар Но.","Склад-приемник","Номер ЗЛ"]].sort_values(by="Номер ЗЛ",ascending = False)
zab.columns = ['СерийныйНомер','ТоварНо','КодСклада','ЗЛ']

#делим файл на две части:необиркованные и без СН на машине
zab_0=zab[zab.СерийныйНомер.isnull()]
zab_0=zab_0[['КодСклада','ТоварНо']]
zab=zab[zab.СерийныйНомер.str.contains('СН',na=False)]
zab=zab[['СерийныйНомер','КодСклада']]

#проставляем магаз в серийники, которые на 4001/3075/опте
df=pd.merge(ost_obr1,zab,on=['СерийныйНомер'],how = 'inner')#inner join оставит только неперемещенный товар
ost_obr1=df[['КодСклада','ТоварНо']]

#соединяем остатки на магазах,на машине, то что числилось на 4001/3075/опте
result = pd.concat([ost_obr,ost_obr1,zab_0])

#догружаем размер/артикул/толщ.плетения и тд
df=pd.read_excel(r'C:\анаконда\карточки.xlsx',sheet_name='обручи',header=6)
result=pd.merge(result,df,left_on=['ТоварНо'],right_on=['Номер'],how='left')
del result['Номер']
result.columns=[x.replace(' ','_') for x in result.columns]


In [18]:
a1513 = result[result.Товарная_подгруппа == 1513]
a1514 = result[result.Товарная_подгруппа == 1514]
a1515 = result[result.Товарная_подгруппа == 1515]
a1516 = result[result.Товарная_подгруппа == 1516]

In [14]:
a1514 = 
a1515 = 
a1516 =

,КодСклада,ТоварНо,Дизайн,Описание,Артикул_товара,Толщина_плетения,Товарная_подгруппа,Размер
0,5,ТОВ0963682,ИФ ОБРУЧ ГЛАДКАЯ КОМФОРТ,Кольцо обручальное,КОМ 05-00,5.0,1522,20.0
1,5,ТОВ0832938,ИФ ОБРУЧ ГЛАДКАЯ,Кольцо обручальное,КО 05-00,5.0,1516,16.5
2,5,ТОВ0877934,ИФ ОБРУЧ ГЛАДКАЯ,Кольцо обручальное,КО 02-00,2.0,1513,15.0
3,5,ТОВ0866933,ИФ ОБРУЧ АЛМ ГР ЛЮКС,Кольцо обручальное,23-08-0000-08600,6.0,1525,20.0
4,5,ТОВ0832964,ИФ ОБРУЧ ГЛАДКАЯ,Кольцо обручальное,КО 04-00,4.0,1515,17.5


In [ ]:
#сохраняем в эксель. нужно доработать сохранение в уже существующий эксель
with pd.ExcelWriter(r'C:\Остатки\обручи.xlsx') as writer:
    result.to_excel(writer, sheet_name='остатки')